## Scikit-Learn é uma biblioteca que auxilia na validação do modelo.

A separação de dados em treino e teste é muito importante para ter-se condições de avaliar o desempenho de um modelo em dados que ainda não foram vistos. Na verdade, dados ainda não vistos são o foco de aprendizado de máquina, visto que ao desenvolver-se sistemas, o principal objetivo é colocá-los em produção e ter performance aceitável (confiável).

Existe uma subdivisão dos dados de treino que pode ser o conjunto de validação. O conjunto de validação pode, por exemplo, participar de um processo de melhoria do modelo. Uma vez que o modelo está aperfeiçoado suficiente, então ele é avaliado no conjunto de teste.

Vamos explorar um conjunto de dados diferentes: uma base de vinhos; separar os dados e verificar como a mudança de parâmetros afeta o resultado. Há uma forma automatizada de fazer isso, mas isso será conteúdo do próximo encontro. Por enquanto, vamos fazer tudo manualmente para entender o processo.

In [ ]:
from sklearn.datasets import load_wine
wine = load_wine()

data = wine.data
target = wine.target

In [ ]:
print(data.shape)

In [ ]:
# separando os dados em treino e teste
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.33, random_state=42)

# separando o conjunto de treino em validação também
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.33, random_state=42)

Agora, os dados estão divididos em três blocos disjuntos: treino, validação e teste. 
* O teste representa aproximadamente 33% da quantidade dados;
* A validação representa aproximadamente 22% (66% x 33%) do total de dados; e
* O treino representa aproximadamente 44% da quantidade de dados originais;

A seguir, será comparado o desempenho do KNN treinado com o conjunto de treino e testado no conjunto de validação.

In [ ]:
# treinando o modelo 
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)

# avaliando o modelo
from sklearn.metrics import accuracy_score
y_pred = knn.predict(X_val)
accuracy_score(y_val, y_pred)

O resultado mostra a princípio como a redução na quantidade de dados de treino impacta o desempenho do método. Esse conjunto de validação é bastante importante para servir como um tipo de teste e analisar a relação de melhoria ou piora com a mudança de parâmetros no método de aprendizagem.

Vamos supor testes com diferentes valores de *k* (n_neighbors) no método k-NN. Vamos mostrar o desempenho a cada mudança de parâmetro, e também vamos salvar o modelo sempre que um resultado maior for obtido.

In [ ]:
best_model = None
best_accuracy = 0

for k in [1,2,3,4,5]:

    knn = KNeighborsClassifier(n_neighbors = k) # a cada passo, o parâmetro assume um valor
    knn.fit(X_train, y_train)

    y_pred = knn.predict(X_val)
    acc = accuracy_score(y_val, y_pred)
    print('K:', k, '- ACC:', acc)
    
    if acc > best_accuracy:
        best_model = knn
        best_accuracy = acc
        
y_pred = best_model.predict(X_test)
acc = accuracy_score(y_test, y_pred)

print()
print('Melhor modelo:')
print('K:', best_model.get_params()['n_neighbors'], '- ACC:', acc * 100) #corrigir

No trecho de código anterior, foi avaliado o desempenho do método de aprendizagem usando 5 parametrizações diferentes: k = 1, 2, 3, 4 e 5. Em vez de fazer essa verificação diretamente no conjunto de teste, essa verificação deve ser feita no conjunto de validação quando houver dados suficiente. No exemplo anterior, o melhor resultado foi obtido com k = 1. Assim, o modelo salvo como *melhor modelo* foi k = 1, e foi esse modelo o escolhido para ser utilizado na verificação de performance com o conjunto de teste.

Agora que o conjunto de teste está totalmente isolado e não é usado para definir melhores parâmetros, é importante se atentar para outra questão: suponha que, por acaso, as amostras mais fáceis tenham caído no conjunto de teste. Isso vai influenciar o resultado positivamente, gerando uma falsa impressão de que o método de aprendizagem trouxe bons resultados. Aqui duas saídas são possíveis: embaralhar os dados e rodar o mesmo experimento várias vezes ou utilizar validação cruzada.

**Os experimentos feitos até aqui usaram a estratégia de validação hold-out**, onde uma parcela dos dados é isolada e utilizada para verificar o desempenho do modelo. Na primeira proposta para contornar o problema de amostras muito fáceis caírem no conjunto de teste, sugeriu-se usar várias vezes o hold-out com embaralhamento das amostras. Embora aplicável, estatisticamente existe um grau de cetismo da viabilidade disso. Isso porque não há garantias de que todas as amostras em algum momento estarão no conjunto de teste. *A solução de garantia é a validação cruzada ou cross-validation.*

In [ ]:
# embaralhando os dados várias vezes e re-executando o experimento
import numpy as np
from sklearn.model_selection import ShuffleSplit
ss = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0) # 5 execuções diferentes com 20% dos dados para teste

acc = []
for train_index, test_index in ss.split(data):
    knn = KNeighborsClassifier(n_neighbors = 1)
    knn.fit(data[train_index],target[train_index])
    y_pred = knn.predict(data[test_index])
    acc.append(accuracy_score(y_pred,target[test_index]))

acc = np.asarray(acc) # converte pra numpy pra ficar mais simples de usar média e desvio padrão
print(acc)
print('Acurácia - %.2f +- %.2f' % (acc.mean() * 100, acc.std() * 100))

Aqui é importante perceber a diferença da acurácia entre essa validação (repetindo 5 vezes o experimento) e o bloco de código anterior. Na situação anterior, o modelo de k-NN com k = 1 chegou próximo a 78% de acurácia, enquanto que ao repetir o experimento, a média foi 71,11% com desvio de 8.89%, que é bastante coisa. **Pode-se dizer que é mais confiável afirmar que esse modelo tem acurácia em torno de 71% do que 78%.**

A validação cruzada pode ser feita de várias formas no Scikit-Learn, mas é importante observar que agora não temos mais apenas um resultado para o desempenho do modelo. Teremos um resultado para cada execução, então é necessário olhar para essas informações pela perspectiva da média e desvio padrão. As funções do Scikit-Learn que auxiliarão na validação cruzada são: *KFold* e *cross_val_score*.

In [ ]:
# utilizando validação cruzada com cross_val_score
from sklearn.model_selection import cross_val_score
knn = KNeighborsClassifier(n_neighbors = 1)
scores = cross_val_score(knn, data, target, cv=5) # 5 execuções diferentes com 20% dos dados para teste

print('Acurácia - %.2f +- %.2f' % (scores.mean() * 100, scores.std() * 100))

In [ ]:
# utilizando validação cruzada com KFold
from sklearn.model_selection import KFold
kf = KFold(n_splits = 5)

acc = []
for train_index, test_index in kf.split(data):
    knn = KNeighborsClassifier(n_neighbors = 1)
    knn.fit(data[train_index],target[train_index])
    y_pred = knn.predict(data[test_index])
    acc.append(accuracy_score(y_pred,target[test_index]))

acc = np.asarray(acc) # converte pra numpy pra ficar mais simples de usar média e desvio padrão
print(acc)
print('Acurácia - %.2f +- %.2f' % (acc.mean() * 100, acc.std() * 100))

Os dois blocos de cima demonstram duas maneiras de aplicar a validação cruzada a um modelo. Enquanto o primeiro afirma que a acurácia do modelo é em torno de 72%, o segundo alega 63% com desvio de 23% (que é muito alto). O primeiro tem o objetivo de avaliar o desempenho baseado em uma métrica, enquanto o segundo oferece quais os índices (linhas) que participarão de cada rodada ou execução, dando maior flexibilidade para o programador.

**Mas por que uma diferença tão grande?** A resposta é um conceito do campo de estatística: estratificação. Ter um conjunto estratificado significa que ele é representativo. Ou seja, se sua amostra tem 30 amostras da classe positiva e 30 da classe negativa, ao separar em um conjunto menor, é esperado que mantenha-se essa proporção (50/50). Enquanto que o cross_val_score faz a divisão considerando a estratificação, o KFold não o faz. Para isso, existe uma variante dessa função chamada StratifiedKFold. 

In [ ]:
# utilizando validação cruzada com KFold
from sklearn.model_selection import StratifiedKFold
kf = StratifiedKFold(n_splits = 5)

acc = []
for train_index, test_index in kf.split(data, target): # precisa passar as classes agora para que a divisão aconteça
    knn = KNeighborsClassifier(n_neighbors = 1)
    knn.fit(data[train_index],target[train_index])
    y_pred = knn.predict(data[test_index])
    acc.append(accuracy_score(y_pred,target[test_index]))

acc = np.asarray(acc) # converte pra numpy pra ficar mais simples de usar média e desvio padrão
print('Acurácia - %.2f +- %.2f' % (acc.mean() * 100, acc.std() * 100))

Para esse método de aprendizagem, o conjunto de dados wine parece ser mais difícil de classificar. Diferente da Iris, que ficava em torno de 98%, esse conjunto dificilmente passou dos 80%.
Vamos aplicar a visualização desses dados para verificar se eles são muito aglomerados ou linearmente separáveis.

In [ ]:
from sklearn.decomposition import PCA

import matplotlib.pyplot as plt

pca = PCA(n_components=2)
X_r = pca.fit_transform(data)

colors = ['navy', 'turquoise', 'darkorange']
for color, i, target_name in zip(colors, [0, 1, 2], wine.target_names):
    plt.scatter(X_r[target == i, 0], X_r[target == i, 1], color=color, alpha=.8, label=target_name)
plt.legend(loc='best', shadow=False, scatterpoints=1)
plt.title('PCA')

Os dados estão bastante aglomerados em algumas regiões, dificultando o funcionamento de métodos de aprendizagem baseados em distância e funções lineares. Os dados naturalmente têm essa característica, mas é sempre importante verificar se essa aglomeração de informação não é devido a escala dos atributos. Isso, inclusive, é extremamente prejudicial para métodos baseados em distância.

In [ ]:
# verificando a escala dos atributos
import pandas as pd
df = pd.DataFrame(data)
df.describe()

A biblioteca Pandas tem uma forma fácil de visualizar algumas informações estatísticas sobre um conjunto de dados. No bloco acima, é possível perceber que alguns atributos estão em escalas completamente diferente de outros. Por exemplo, compare o atributo 0 e o atributo 12, ou 0 e 7. Escalas diferentes confundem os métodos baseados em distância. Sabendo disso, uma estratégia é colocar os dados para respeitar um padrão ou distribuição. Isso pode ser feito com o StandardScaler do Scikit-Learn. Vamos verificar como fica a distribuição dos dados após padronizá-los e a visualização em 2D.

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt

scaler = StandardScaler()
scaler.fit(data)
data_s = scaler.transform(data)

df = pd.DataFrame(data_s)
df.describe()

In [ ]:
pca = PCA(n_components=2)
X_r = pca.fit_transform(data_s)

colors = ['navy', 'turquoise', 'darkorange']
for color, i, target_name in zip(colors, [0, 1, 2], wine.target_names):
    plt.scatter(X_r[target == i, 0], X_r[target == i, 1], color=color, alpha=.8, label=target_name)
plt.legend(loc='best', shadow=False, scatterpoints=1)
plt.title('PCA')

Os dados agora estão todos muito próximos de 0 com desvio padrão 1, e isso melhora a disposição gráfica como pode ser visto na imagem. Não há mais tanto aglomeração, e existe separações lineares mais visíveis. Isso são fortes indícios de que agora um método de aprendizagem terá melhor desempenho. No entanto, enquanto para visualizar o StandardScaler foi aplicado em toda a base, é importante perceber que ao testar o modelo, **o StandardScaler só pode ter dados de treino no fit**. Ou seja, ele não pode conhecer a disposição dos dados de teste.

In [ ]:
from sklearn.model_selection import StratifiedKFold
kf = StratifiedKFold(n_splits = 5)

acc = []
for train_index, test_index in kf.split(data, target): # precisa passar as classes agora para que a divisão aconteça
    knn = KNeighborsClassifier(n_neighbors = 1)
    
    scaler = StandardScaler()
    train = scaler.fit_transform(data[train_index]) # somente dados de treino no fit
    test = scaler.transform(data[test_index]) # aplica-se transform no teste apenas
    
    knn.fit(train,target[train_index])
    y_pred = knn.predict(test)
    acc.append(accuracy_score(y_pred,target[test_index]))

acc = np.asarray(acc) # converte pra numpy pra ficar mais simples de usar média e desvio padrão
print('Acurácia - %.2f +- %.2f' % (acc.mean() * 100, acc.std() * 100))

Os resultados melhoraram substancionalmente e são confiáveis, visto que foram avaliados numa estratégia de validação cruzada com 5 partições.

Um último passo é integrar na validação cruzada a pesquisa pelos melhores parâmetros, algo semelhante ao que foi feito anteriormente. Novamente, o Scikit-Learn tem uma função para facilitar o processo chamada *GridSearchCV*. A dificuldade nesse ponto aparece ao perceber como será formado o fluxo, afinal não pode-se procurar os melhores parâmetros a cada execução da validação cruzada, uma vez que a validação cruzada tem o objetivo de avaliar um modelo e a mudança de parâmetro a cada execução criaria um modelo novo. Isso criaria um loop infinito de tentativas de avaliar um modelo.

A solução é separar um conjunto de dados para fazer a busca pelos melhores parâmetros e formar um modelo. Assumindo que esse conjunto utilizado para esse propósito é representativo, avalia-se então com a validação cruzada qual o desempenho desse modelo. 

In [ ]:
# utilizando validação cruzada com cross_val_score
from sklearn.model_selection import cross_val_score

from sklearn.pipeline import Pipeline
pipeline = Pipeline([('scaler', StandardScaler()), ('clf', KNeighborsClassifier(n_neighbors = 1))])
scores = cross_val_score(pipeline, data, target, cv=5) # 5 execuções diferentes com 20% dos dados para teste

print('Acurácia - %.2f +- %.2f' % (scores.mean() * 100, scores.std() * 100))

In [ ]:
# separa-se uma parcela para encontrar os melhores parâmetros (5% do original)
data_gs, data_cv, target_gs, target_cv = train_test_split(data, target, test_size=0.95, random_state=42, stratify=target)

# uma forma automática de StandardScaler + CLF
from sklearn.pipeline import Pipeline
pipeline = Pipeline([('scaler', StandardScaler()), ('clf', KNeighborsClassifier())])

# utiliza-se GridSearchCV para achar os melhores parâmetros
from sklearn.model_selection import GridSearchCV
parameters = {'clf__n_neighbors': [1,2,3,4,5], 'clf__weights' : ['uniform','distance']} # quais parâmetros e quais valores serão testados
clf = GridSearchCV(pipeline, parameters, cv=3, iid=False) # clf vai armazenar qual foi a melhor configuração
clf.fit(data_gs, target_gs)

print(clf.best_params_)

# utilizando validação cruzada para avaliar o modelo
scores = cross_val_score(clf.best_estimator_, data_cv, target_cv, cv=5)
print('Acurácia - %.2f +- %.2f' % (scores.mean() * 100, scores.std() * 100))

clf = clf.best_estimator_

from sklearn.model_selection import StratifiedKFold
kf = StratifiedKFold(n_splits = 5)

acc = []
for train_index, test_index in kf.split(data_cv, target_cv): # precisa passar as classes agora para que a divisão aconteça
    
    #scaler = StandardScaler()
    #train = scaler.fit_transform(data_cv[train_index]) # somente dados de treino no fit
    #test = scaler.transform(data_cv[test_index]) # aplica-se transform no teste apenas
    
    clf.fit(data_cv[train_index],target_cv[train_index])
    y_pred = clf.predict(data_cv[test_index])
    acc.append(accuracy_score(y_pred,target_cv[test_index]))

acc = np.array(acc)
print('Acurácia - %.2f +- %.2f' % (acc.mean() * 100, acc.std() * 100))

O resultado melhorou ainda mais ao pesquisar qual era a melhor configuração de atributos, antes de avaliar a performance do modelo no conjunto de teste. O conceito e a função de Pipeline serão abordados com mais detalhes nos próximos encontros, então não se desespere.

Conclui-se aqui essa breve discussão e apresentação sobre divisão de dados, onde você pode ter aprendido sobre:
* separação dos dados em treino, validação e teste;
* normalização dos dados;
* validação cruzada; e
* busca pelos melhores parâmetros.

# Exercício

* Com o conjunto de dados sobre *câncer de mama*, **tente obter o melhor desempenho de acurácia**. 

* Organize e **tenha cuidado** para que seu experimento execute um *protocolo de validação que faça sentido*.

Mais informações sobre esse conjunto de dados poderão ser obtidas em: 
[https://scikit-learn.org/stable/datasets/index.html#breast-cancer-dataset](https://scikit-learn.org/stable/datasets/index.html#breast-cancer-dataset)

In [ ]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

breast_cancer = load_breast_cancer()
X = breast_cancer.data
y = breast_cancer.target